In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

2023-11-22 18:45:03.705014: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 18:45:03.753147: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 18:45:03.753285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 18:45:03.755306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 18:45:03.762269: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 18:45:03.762934: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
tf.__version__

'2.15.0'

In [3]:
#!wget https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

In [4]:
model = keras.models.load_model('bees-wasps.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

MODEL_FILENAME = 'bees-wasps.tflite'

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpumla99b4/assets


INFO:tensorflow:Assets written to: /tmp/tmpumla99b4/assets
2023-11-22 18:45:09.112381: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-22 18:45:09.112407: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-22 18:45:09.112895: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpumla99b4
2023-11-22 18:45:09.113808: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-22 18:45:09.113820: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpumla99b4
2023-11-22 18:45:09.117060: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-22 18:45:09.119049: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-22 18:45:09.245367: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [5]:
!du -sh 'bees-wasps.tflite'

43M	bees-wasps.tflite


### Q2

In [6]:
import tensorflow.lite as tflite

In [7]:
interpreter = tflite.Interpreter(model_path=MODEL_FILENAME)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

input_index, output_index

(0, 13)

### Q3

In [9]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [10]:
IMG_URL = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
TARGET_SIZE =(150, 150)

In [11]:
IMG = download_image(IMG_URL)

In [12]:
def prepare_input(x):
    return x / 255.0

In [43]:
img = prepare_image(IMG, TARGET_SIZE)

In [44]:
x = np.array(img, dtype='float32')

In [45]:
X = np.array([x])

In [46]:
X = prepare_input(X)
X[0,0,0,0]

0.94509804

### Q4

In [47]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [48]:
preds

array([[0.65898407]], dtype=float32)